In [1]:
import re
import time
import pytesseract
from PIL import Image
import pandas as pd
from datetime import datetime
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
home_page_url= "https://services.ecourts.gov.in/ecourtindia_v6/"
cause_list_id= "leftPaneMenuCL"
close_btn_cl_class= "btn-close"
crn = "MHAU019999992015"
state_id= "sess_state_code"
dist_id= "sess_dist_code"
court_complex_id= "court_complex_code"
court_name_id= "CL_court_no"
pop_up_x= "//button[contains(@class,'btn-close')]"

In [3]:
captcha_id= "captcha_image"
court_name_id= "CL_court_no"
to_fill_captcha_id= "cause_list_captcha_code"

In [4]:
criminal_path= "//button[@onclick=\"submit_causelist('cri')\"]"
civil_path= "//button[@onclick=\"submit_causelist('civ')\"]"

In [5]:
def close_pop_up(driver):
    try:
        wait = WebDriverWait(driver, 10)
        close_btn = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(@class,'btn-close')]")
            )
        )
        close_btn.click()
        print("Popup closed!")
    except Exception as e:
        print("Close button not clickable:", e)

In [6]:
def get_dropdown(driver, id_= "sess_state_code", name= "West Bengal"):
    dropdown = driver.find_element(By.ID, id_)
    select= Select(dropdown)
    options= select.options
    for opt in options:
        # print(opt.text, " --> ", opt.get_attribute("value"))
        pass
    # select states
    select.select_by_visible_text(name)
    print(f"selected {name}")

In [7]:
def fill_captcha(driver, captcha_id=captcha_id, captcha_to_fill_id= to_fill_captcha_id):
    captcha_element= driver.find_element(By.ID, captcha_id)
    captcha_element.screenshot("captcha.png")
    captcha= Image.open("captcha.png")
    captcha_text= pytesseract.image_to_string(captcha).strip()
    cleaned_captcha= re.sub(r'[^a-zA-Z0-9]', '', captcha_text)
    print(cleaned_captcha)
    f_captcha= driver.find_element(By.ID, captcha_to_fill_id)
    f_captcha.clear()
    f_captcha.send_keys(cleaned_captcha)

In [8]:
def click_on_button(driver, xpath=criminal_path):
    criminal_btn = driver.find_element(By.XPATH, xpath)
    criminal_btn.click()
    print("form submitted successfully!")

In [9]:
def validate_date(date_):
    try:
        datetime.strptime(date_, "%d-%m-%Y")
        print("date validated successfully!")
        return True
    except:
        print("date format validation failed!")
        raise ValueError("Invalid date format, expected DD-MM-YYYY")

In [10]:
def put_causelist_date(driver, date_=None,datefield_id="causelist_date"):
    try:
        if date_:
            date_field= driver.find_element(By.ID, datefield_id)
            date_field.clear()
            date_field.send_keys(date_)
            print("cause list date has been filled!")
    except Exception as e:
        print(f"Something went wrong!")
        print(e)

In [11]:
def get_table_content(driver, disp_table="dispTable"):
    data_list = []

    try:
        # Try to locate the table by ID
        try:
            table = driver.find_element(By.ID, disp_table)
        except NoSuchElementException:
            print("Table unavailable!")
            return "Table not found"

        # Locate rows (skip first 2 header rows)
        rows = table.find_elements(By.TAG_NAME, "tr")[2:]

        for row in rows:
            cols = row.find_elements(By.TAG_NAME, 'td')
            if len(cols) >= 4:
                sr_no = cols[0].text.strip()

                # Extract case number (last line from the cell)
                case_text = cols[1].text.strip()
                case_number = case_text.split("\n")[-1]

                party_name = cols[2].text.replace("\n", " ").strip()
                advocate = cols[3].text.strip()

                data_list.append({
                    "sr_no": sr_no,
                    "case_number": case_number,
                    "party_name": party_name,
                    "advocate": advocate
                })
        df= pd.DataFrame(data_list)
        print(df.head())
        return df

    except Exception as e:
        print("Error while scraping:", e)
        return None

In [12]:
# Initiliaze driver and open homepage
date_ = "21-10-2025"
case_type="criminal"
driver= Chrome()
driver.get(home_page_url)
time.sleep(5)
# goto cause list page
cause_list= driver.find_element(By.ID, cause_list_id)
cause_list.click()
time.sleep(2)
# close popup in causelist
close_pop_up(driver)
time.sleep(2)
# select state
get_dropdown(driver=driver)
time.sleep(2)
# select district
get_dropdown(driver=driver, id_=dist_id, name="Paschim Bardhaman")
time.sleep(2)
# select court complex
get_dropdown(driver=driver, id_=court_complex_id, name="ASANSOL COURT COMPLEX")
time.sleep(2)
# select court name
get_dropdown(driver=driver, id_= court_name_id, name="9-Indrani Gupta-CJM")
# fill captcha
fill_captcha(driver)
time.sleep(2)
# fill date
validate_date(date_=date_)
put_causelist_date(driver, date_=date_)
time.sleep(2)
# fill captcha
if case_type == "criminal":
    click_on_button(driver, xpath=criminal_path)
elif case_type == "civil":
    click_on_button(driver, xpath=civil_path)
time.sleep(2)
# get table content
df= get_table_content(driver)
# close the driver
time.sleep(2)
driver.quit()

Popup closed!
selected West Bengal
selected Paschim Bardhaman
selected ASANSOL COURT COMPLEX
selected 9-Indrani Gupta-CJM
3ck7fy
date validated successfully!
cause list date has been filled!
form submitted successfully!
  sr_no             case_number  \
0     1   ViewM.V .CASE/31/2023   
1     2   ViewM.V .CASE/39/2023   
2     3   ViewM.V .CASE/82/2023   
3     4  ViewM.V .CASE/191/2023   
4     5  ViewM.V .CASE/230/2023   

                                      party_name advocate  
0       State of West Bengal versus amit kr shaw           
1      State of West Bengal versus sachin kr ram           
2  State of West Bengal versus BUDDHA CHATTERJEE           
3     State of West Bengal versus MANOWAR ANSARI           
4       State of West Bengal versus RABIN RAMANI           


In [13]:
df.head()

,sr_no,case_number,party_name,advocate
0,1,ViewM.V .CASE/31/2023,State of West Bengal versus amit kr shaw,
1,2,ViewM.V .CASE/39/2023,State of West Bengal versus sachin kr ram,
2,3,ViewM.V .CASE/82/2023,State of West Bengal versus BUDDHA CHATTERJEE,
3,4,ViewM.V .CASE/191/2023,State of West Bengal versus MANOWAR ANSARI,
4,5,ViewM.V .CASE/230/2023,State of West Bengal versus RABIN RAMANI,


In [14]:
df.to_json("cause_list.json",index=False)